# Why Not? - Generative Adversarial Networks


## What are Gans - wikipedia

Generative adversarial networks (GANs) are a class of artificial intelligence algorithms used in unsupervised machine learning, implemented by a system of two neural networks contesting with each other in a zero-sum game framework. They were introduced by Ian Goodfellow et al. in 2014. This technique can generate photographs that look at least superficially authentic to human observers, having many realistic characteristics (though in tests people can tell real from generated in many cases)

More at : https://en.wikipedia.org/wiki/Generative_adversarial_network


## Intuition

![fakeWatches](resources/fake_watches.png)

As people start identifying the fake-watch seller's watch to be fake, he improves them. Then better spotters identify the watches as fake, and he improves them further. This keeps going on till, at some point, there is nothing different between a fake watch and a real watch

1. Learn a simple real vs fake classifier - with a small dataset
2. Reward a generator to fake samples
3. Reward discriminator to discriminate between fake and real.
4. Keep repeating this


## Let's get started 

### The Model View
![GAN MODEL](resources/gan_model.jpeg)


### The Library

* Keras : Gives a very high level view of neural networks. 
* You can add and stack layers to your model easily
* We will use the Keras + Tensorflow combination



### First steps - Imports
Lets also start a class definition. We will define the class in parts, and add some jupyter specific code

In [9]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator() # Some Magical function
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator() # Another Magical Function
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid) # Lets connect the two models
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

### Generator
* Now that we have our general flow chart ready, let's start implementing those functions.
* For the generator, we ideally want to start off with noise and generate an image.  
* We stack a bunch of convolutional layers together, and do some upsampling.

In [10]:
def build_generator(self):

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_shape=noise_shape))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8)) 
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

DCGAN.build_generator = build_generator #For jupyter -> disjointed classes

### Discriminator

* Now that we have the structure and a generator, we need to build a discriminator which discriminates real from fake
* We want to start with an image and output a real/fake decision
* We stack a bunch of convolutional layers, starting from image dimensions, giving out a value (0/1 - real/fake)


In [11]:
def build_discriminator(self):

    img_shape = (self.img_rows, self.img_cols, self.channels)

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

DCGAN.build_discriminator = build_discriminator # For jupyter - disjointed classes

### Training loop
* Now we have the two elements - A generator and a discriminator
* It's time to run the training loop
* Dataset : Mnist (A simple digits based dataset)
* Batch Size : 128 by default
* Pre-Steps
    1. Load Data
    2. Normalize Data
* Steps
    1. Select randomly some images from the dataset (0.5 times the batch size)
    2. Generate the remaining(0.5 times batch size) images from noise
    3. Train the discriminator
        a. Real Images from dataset as 1
        b. Fake images from generator as 0
    4. Train the generator 
        a. Input is a full batch of noise
        b. All these elements are treated as real (=1)


In [12]:
def train(self, epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    half_batch = int(batch_size / 2)

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

        # Sample noise and generate a half batch of new images
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = self.generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, 100))

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            self.save_imgs(epoch)

DCGAN.train = train #jupter notebook class hack

### Let's run the entire loop

And add in a small utility function to cache generated images every 50 batches


In [13]:
def save_imgs(self, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = self.generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    #fig.suptitle("DCGAN: Generated digits", fontsize=12)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("resources/images/mnist_%d.png" % epoch)
    plt.close()
    
DCGAN.save_imgs = save_imgs

if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8, 64)          0         
__________

### Results

#### So in the beginning we have shit :-
![MNIST_0](resources/images/mnist_0.png)

#### Then, it get's less shittier (epoch 1000): 
![MNIST_1000](resources/images/mnist_1000.png)


#### Till shit get's awesome (epoch 7950)
![MNIST_8000](resources/images/mnist_7950.png)

#### All together
![MNIST_Animated](resources/images/mnist_anim.gif)